# Getting started with Amazon Braket Hybrid Job
This tutorial shows how to run a simple Amazon Braket Hybrid Job. There are two pieces of code required for every Braket Job: an algorithm script, and a job creation script. The creation script will be shown below. The algorithm script can be a separate file or a Python module. To get started quickly, we consider simple circuits with only one qubit and one gate in this example notebook.

## Learning Outcomes 
* Run our first Braket Job! 
* Understand how to run scripts or functions
* Use Braket Jobs within a specific AWS session 
* Use sync or async mode
* Use Local Braket Jobs to quickly test code [Docker required]
* Monitor job status and view logs 
* Save results 

## Algorithm script

The algorithm script for this notebook is [here](algorithm_script.py) as a separate file. The code block below is a copy of the algorithm script. As shown, each of our circuits has only one $X$ rotation gate with a random angle. The circuit is repeated five times with different random rotations. We would write the algorithm script as we usually do, except that we do not specify the backend QPUs or simulators explicitly. Instead, it is provided through environment variables which are passed to the algorithm script when creating a Braket Job. 


In [1]:
%%script echo "This block is a copy of the algorithm script."

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result


print("Test job started!")

# Use the device declared in the Orchestration Script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list})

print("Test job completed!")

This block is a copy of the algorithm script.


## Braket Job Creation script
In the creation script, we use <code>AwsQuantumJob.create</code> to create a Braket job. When the Braket job is created, it starts an EC2 instance and spins up a Docker container. The instance type, container and other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks for more information.

The required inputs of <code>AwsQuantumJob</code> are:
- <b>device_arn</b>: The arn of the Braket simulator or QPU we want to use. It will be stored as an environment variable for the algorithm script.
- <b>source_module</b>: The path to a file or a python module that contains your algorithm script. It will be uploaded to the container for Braket Job execution.
- <b>entry point</b>: The path relative to the source_module. It points to the piece of code to be executed when the Braket Job starts.
- <b>wait_until_complete (optional)</b>: If true, the function call will wait until the Braket Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. 

In [14]:
from braket.aws import AwsQuantumJob

In [15]:
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    "arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    wait_until_complete=True,
)

KeyboardInterrupt: 

In this exercise, the algorithm script is in one file, so we set <code>source_module</code> to be <code>algorithm_script.py</code> and <code>entry_point</code> to be <code>algorithm_script</code>. Depending on your application, there are other options for setting the algorithm script. For example, if you wish to only execute a part of <code>algorithm_script.py</code> at the start of a Braket Job, you can package that part to be a <code>starting_function()</code>. Then the input arguments would be

In [4]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

When your algorithm script requires other files such as helper functions, you can put them all in one folder, say the <code>algorithm_folder</code>. The input arguments would then be

In [5]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

# AWS Sessions

We can customize where Braket Jobs saves and loads results in Amazon S3 by providing the AWS session information. 


In [13]:
from braket.aws import AwsSession

aws_session = AwsSession(default_bucket = "amazon-braket-{region}-{account_id}") # Set Amazon S3 bucket

To create a Braket job with this S3 bucket, we pass `aws_session` as an argument to ` AwsQuantumJob.create()`

In [18]:
aws_session = aws_session # using specific S3 bucket

## Local Job
Braket Job also provides an option to test the job locally by using LocalQuantumJob. This may let's you locally test code before submitting to Amazon Braket Job. This feature requires Docker to be installed. Amazon Braket Notebook Instances already have Docker pre-installed [Amazon Braket notebook instances](https://docs.aws.amazon.com/braket/latest/developerguide/braket-get-started-create-notebook.html), so you can test Local jobs within a NBI. To install Docker in your local environment, follow these [instructions](https://docs.docker.com/get-docker/).

In [6]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    "arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script"
)

Using the short-lived AWS credentials found in session. They might expire while running.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


Boto3 Version:  1.18.33
Beginning Setup
Getting setup script from  s3://braket-external-assets-preview-us-west-2/HybridJobsAccess/scripts/setup-container.sh
download: s3://braket-external-assets-preview-us-west-2/HybridJobsAccess/libraries/amazon-braket-schemas-python.tar.gz to ../../braket/additional_lib/original/amazon-braket-schemas-python.tar.gz
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.coveragerc
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/bug_report.md
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/config.yml
aws-amazon-braket-schemas-python-staging-6ffc1f24d0886b62d721a8c99fe2366d1f62822e/.github/ISSUE_TEMPLATE/documentation_req

## Results
The status of a Braket Job can be checked by calling <code>job.state()</code>. Once completed, the results are stored in Braket Job and can be retrieved using <code>job.result()</code>. Logs and metadata are also stored in a Braket Job. If you lose the job variable, you can always retrieve it by its arn which you can find in the console.

In [7]:
results = job.result()
print("counts")
print(results["counts"])
print("angles")
print(results["angles"])

counts
[{'0': 81, '1': 19}, {'0': 99, '1': 1}, {'1': 86, '0': 14}, {'1': 74, '0': 26}, {'0': 60, '1': 40}]
angles
[1.0709881821004101, 0.2694177348393929, 3.947298780847625, -1.926608246444919, -1.316898137114331]


In [8]:
%%capture captured
print(job.logs())
print(job.metadata())

The job result can also be downloaded to the current directory by

In [9]:
job.download_result()

## Summary
In this tutorial, we have prepared an algorithm and an orchestration script for a simple example. We have learned how to create a Braket Job and how to retrieve the results once the job is completed.